# Пример расчета длительности эксперимента и mde (на основе доступной выборки)

In [14]:
import io
import sqlalchemy
from sqlalchemy import create_engine
from clickhouse_driver.client import Client
import pandas as pd
import numpy as np
from itertools import combinations, product
import scipy.stats as st
import math 
from o3_exp import o3_exp
from datetime import datetime, date, time
import matplotlib.pyplot as plt
import plotly.express as px
import json

In [ ]:
# подключение к clickhouse
client_click = get_creds(ch)

In [1]:
# чтение данных из clickhouse
def get_data(client, query, date_start, date_end, user_col, smpl, columns):
    date_range  = pd.date_range(start=date_start,end=date_end)
    df_arr = []
    for date in date_range:
        for part in range(0,smpl):
            date_str = date.strftime('%Y-%m-%d')
            temp = client.execute(query.format(date=date_str, user_col=user_col, sample_value=1/smpl, offset_value=(1/smpl*part)))
            df_temp = pd.DataFrame.from_records(temp, columns = columns)
            print(date_str, df_temp.shape,1/smpl,part,(1/smpl*part))
            df_arr.append(df_temp)
    df = pd.concat(df_arr)
    return df

In [13]:
DATE_START = '2021-01-01'
DATE_END = '2021-01-28'
dates = (datetime.strptime(DATE_END, '%Y-%m-%d') - datetime.strptime(DATE_START, '%Y-%m-%d')).days + 1
dates

28

In [7]:
query = '''
with raw as (
    select 
        date 
        , {user_col}
        , sku
        , timestamp
        , action
        , items.sku skus 
        , total_quantity - quantity as result_quantity 
    from tracker.events
    where 1=1 
        and date = '{date}' 
        and namespace = 'bx'
        and user_id != ''
        and action in ('cart_view','decrement', 'increment', 'to_checkout', 'to_cart')
        and page = 'cart'
),
events as (
    select
        date
        , {user_col}
        , flow
        , sku
        , sum(sku_result_quantity) as quantity_diff
    from(
        select
            date
            , {user_col}
            , arrayJoin(arrayReverseSplit(x->x.3 = 'to_checkout', arraySort(groupArray((timestamp, --1
                                                                            sku, --2
                                                                            action, --3
                                                                            skus, --4
                                                                            result_quantity --5
                                                                            ))))) as flow
            , arrayFilter(x -> (x.3 in ('to_checkout')),flow)  as to_checkout_action
            , arrayFlatten(tupleElement(to_checkout_action, 4)) to_checkout_skus
            , length(to_checkout_action) as has_to_checkout
            , arrayJoin(arrayFlatten(arrayFilter(x -> (x.3 in ('decrement','increment','to_cart')),flow))) as change_quantity
            , tupleElement(change_quantity,3) as current_action_type
            , if(current_action_type = 'to_cart', tupleElement(change_quantity,2)*-1, tupleElement(change_quantity,2)) as sku
            , if(current_action_type = 'to_cart', 1 ,tupleElement(change_quantity,5)) as sku_result_quantity
            , has(to_checkout_skus,tupleElement(change_quantity,2)) sku_valid
        from raw
        group by date
            , {user_col} 
    )
    where 1=1
        and has_to_checkout >=1
        and sku_valid = 1
    group by date
        , {user_col}
        , flow
        , sku
),
users_on_cart as (
    select
        date
       , {user_col}
       , 1 as cart_view
   from raw
   where 1=1 
       and action = 'cart_view'
)
SELECT 
    date
    , {user_col}
    , max(cart_view) as cart_view
    , countIf(quantity_diff > 0) as upsale_events_cnt
    , countIf(quantity_diff < 0) as downsale_events_cnt
FROM users_on_cart uoc
    left join events e using(date, {user_col})
GROUP BY  date
    , {user_col}
'''

In [ ]:
df = get_data(client_click, query, DATE_START, DATE_END, 'user_session_id', 10, ['date',
                                                                              'user_client_id',
                                                                              'cart_view',
                                                                              'upsale_events_cnt',
                                                                              'downsale_events_cnt'])
df['date'] = pd.to_datetime(df['date'])
df['date_delta'] = (df['date'] - datetime.strptime(DATE_START, '%Y-%m-%d')).dt.days + 1

In [27]:
def calc_valuable_effect(data, col_num, metric_name):
    result_list = []
    data['date'] = pd.to_datetime(data['date'])
    data['date_delta'] = (data['date'] - datetime.strptime(DATE_START, '%Y-%m-%d')).dt.days + 1
    for n_days in range(7, dates+1, 7):
        user_data = data[(data['date_delta']<=n_days)].groupby('user_client_id')[col_num].sum().reset_index()
        mu = user_data[col_num].mean()
        std = user_data[col_num].std()
        sample_size = user_data.shape[0]
        mde = o3_exp.get_minimal_determinable_effect(mu, std, sample_size, n_groups=2, n_metrics=2, target_share=0.5, r=1, 
                                        compare='only_control', alpha_correction = True, alpha=0.05, beta=0.2)
        result_list.append([metric_name, n_days, mde])
    pd.options.display.float_format='{:.3f}'.format 
    result_df = pd.DataFrame(data = result_list, columns = ['metric_name', 'test_days', 'mde'])
    return(result_df)

In [10]:
calc_valuable_effect(data = df, col_num='upsale_events_cnt',metric_name = 'upsale' )

,metric_name,test_days,mde
0,upsale,7,0.904
1,upsale,14,0.602
2,upsale,21,0.576
3,upsale,28,0.504


In [12]:
calc_valuable_effect(data = df, col_num='downsale_events_cnt',metric_name = 'downsale' )

,metric_name,test_days,mde
0,downsale,7,5.423
1,downsale,14,3.092
2,downsale,21,2.979
3,downsale,28,2.740
